In [2]:
import tkinter as tk
from tkinter import messagebox
import yaml
import os
import pandas as pd
import numpy as np

In [3]:
# Set up list of attributes
attr = ["Armor", 
        "ArmorToughness", 
        "KnockbackResistance", 
        "Health",
        "AttackSpeed",
        "MovementSpeed",
        "AttackSpeed",
        "Damage",
        "Luck"]
attr.sort()
attr_part = ["MainHand", "OffHand", "Head", "Chest", "Legs", "Feet"]
all_attrs = [a + "_" + b for a in attr_part for b in attr]
empty_attrs = dict(zip(all_attrs, [np.nan] * len(all_attrs)))

In [4]:
def parse_attributes(attributes):
    # Flatten the Attributes dict, e.g. Attributes -> MainHand -> Damage: +98
    flat_attrs = empty_attrs.copy()
    if not attributes:
        return flat_attrs
    for hand_or_part, vals in attributes.items():
        for k, v in vals.items():
            flat_attrs[f"{hand_or_part}_{k}"] = v
    return flat_attrs

def parse_enchantments(enchantments):
    # enchantments is a list of strings of the form 'ENCHANTMENT:val'
    if not enchantments:
        return None
    result = {}
    for entry in enchantments:
        if isinstance(entry, str):
            name, val = entry.split(":")
            result[name] = val
    return result

def main():
    # Load in the yml file
    with open("ExampleItems - Copy.yml", "r") as f:
        data = yaml.safe_load(f)

    rows = []
    
    # Iterate over items in the file
    for item_name, item_data in data.items():
        row = {"ItemName": item_name}
        # Basic fields
        row["Id"] = item_data.get("Id")
        row["Data"] = item_data.get("Data", None)

        # Display and Lore (join lore lines into a single string with newlines)
        row["Display"] = item_data.get("Display")
        lore = item_data.get("Lore", [])
        if lore:
            row["Lore"] = "\n".join(lore)
        else:
            row["Lore"] = None

        # Enchantments as a list, then join as "Enchant:Level" strings
        enchants = parse_enchantments(item_data.get("Enchantments"))
        if enchants:
            row["Enchantments"] = ", ".join(f"{k}:{v}" for k,v in enchants.items())
        else:
            row["Enchantments"] = None

        # Attributes flattened
        attrs = parse_attributes(item_data.get("Attributes"))
        row.update(attrs)

        # Options - flatten boolean options if present
        options = item_data.get("Options")
        if options:
            for k, v in options.items():
                row[f"Option_{k}"] = v

        rows.append(row)

    df = pd.DataFrame(rows)

    # Example: save to CSV
    df.to_csv("items_parsed.csv", index=False)
    return df

In [5]:
df = main()
df.head()

,ItemName,Id,Data,Display,Lore,Enchantments,MainHand_Armor,MainHand_ArmorToughness,MainHand_AttackSpeed,MainHand_Damage,...,Feet_ArmorToughness,Feet_AttackSpeed,Feet_Damage,Feet_Health,Feet_KnockbackResistance,Feet_Luck,Feet_MovementSpeed,Option_Unbreakable,Option_Color,Offhand_Damage
0,AdamDefenderofFaith,NETHERITE_SWORD,NaN,&b&lDefender of Faith,"&b&o""You<&sq>ll never break my city or my peop...","DAMAGE_UNDEAD:17, DAMAGE_ARTHROPODS:27, DAMAGE...",23.0,33.0,+30%,98.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AdamSandsofTime,CLOCK,NaN,&b&lSands of Time,&b&oWhy waste time when you could control it?\...,None,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AdamBubblingMass,POPPED_CHORUS_FRUIT,0.0,&b&lStrange Bubbling Mass,&b&o&kaojdwunu ansfcklmonfduaw...,None,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AdamHelmetNether,DIAMOND_HELMET,NaN,&5&lHelm of the Firey Fall,"\n&9[Nether]\n&9[Armor, Helmet]","PROTECTION_ENVIRONMENTAL:62, THORNS:1, PROTECT...",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,NaN
4,AdamGownofHeavensGrace,DIAMOND_LEGGINGS,0.0,&b&lGown of Heaven<&sq>s Grace,&b&oAwarded to the most loyal of all\n&b&oHeav...,"THORNS:21, PROTECTION_PROJECTILE:28, PROTECTIO...",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,NaN


In [6]:
df.columns

Index(['ItemName', 'Id', 'Data', 'Display', 'Lore', 'Enchantments',
       'MainHand_Armor', 'MainHand_ArmorToughness', 'MainHand_AttackSpeed',
       'MainHand_Damage', 'MainHand_Health', 'MainHand_KnockbackResistance',
       'MainHand_Luck', 'MainHand_MovementSpeed', 'OffHand_Armor',
       'OffHand_ArmorToughness', 'OffHand_AttackSpeed', 'OffHand_Damage',
       'OffHand_Health', 'OffHand_KnockbackResistance', 'OffHand_Luck',
       'OffHand_MovementSpeed', 'Head_Armor', 'Head_ArmorToughness',
       'Head_AttackSpeed', 'Head_Damage', 'Head_Health',
       'Head_KnockbackResistance', 'Head_Luck', 'Head_MovementSpeed',
       'Chest_Armor', 'Chest_ArmorToughness', 'Chest_AttackSpeed',
       'Chest_Damage', 'Chest_Health', 'Chest_KnockbackResistance',
       'Chest_Luck', 'Chest_MovementSpeed', 'Legs_Armor',
       'Legs_ArmorToughness', 'Legs_AttackSpeed', 'Legs_Damage', 'Legs_Health',
       'Legs_KnockbackResistance', 'Legs_Luck', 'Legs_MovementSpeed',
       'Feet_Armor', 'Feet_